In [ ]:
import torch as T
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from statistics import mean
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
class Regressor(nn.Module):
    def __init__(self):
        super(Regressor, self).__init__()
        self.input_layer = nn.Linear(8, 64)
        self.hidden1 = nn.Linear(64,32)
        self.hidden2 = nn.Linear(32,8)
        self.output_layer = nn.Linear(8,1)

    def forward(self, x):
        first_layer = self.input_layer(x)
        act1 = F.relu(first_layer)
        second_layer = self.hidden1(act1)
        act2 = F.relu(second_layer)
        third_layer = self.hidden2(act2)
        act3 = F.relu(third_layer)
        out_layer = self.output_layer(act3)
        #x = F.relu(out_layer)
        return out_layer

model = Regressor()
model

In [ ]:
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.classifier.parameters(), lr=0.003)

In [ ]:
epochs = 1
print_every = 40
final_trl = []
final_tel =[]
final_accuracy = []


for e in range(epochs):
    running_loss = 0
    total=0
    correct=0
    train_losses = []
    test_losses = []
    epoch_accurracy =[]
    print(f"Epoch: {e+1}/{epochs}")

    for i, (images, labels) in enumerate(iter(trainloader)):


        
        optimizer.zero_grad()           # Setting gradients to zero
        
        logits = model.forward(images)   # 1) Forward pass
        pred = F.log_softmax(logits, dim=1)
        loss = criterion(pred, labels) # 2) Compute loss
        loss.backward()                  # 3) Backward pass
        optimizer.step()                 # 4) Update model
        
        running_loss += loss.item()

        train_losses.append(loss.item())
        
        
        if i % print_every == 0:
            print(f"\tIteration: {i}\t Loss: {running_loss/print_every:.4f}")
            running_loss = 0

    ### Model evaluation
    model.eval() ## Set model for evaluating.
    with T.no_grad():  ## not upgrade the gradients of the trained model.
        for i, (images, labels) in enumerate(iter(testloader)):

            logits = model.forward(images)  ## Forward pass for prediction
            test_pred = F.log_softmax(logits, dim=1) 

            test_loss = criterion(test_pred, labels) # Compute the loss

            test_losses.append(test_loss.item()) 

            _,predicted = test_pred.max(1)  # Compute accuracy
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            accuracy = 100*correct/total


            epoch_accurracy.append(accuracy)

    model.train() ## Set model to train again.
    
    mean_trl = mean(train_losses)
    mean_tel = mean(test_losses)
    mean_acc = mean(epoch_accurracy)
    final_trl.append(mean_trl)
    final_tel.append(mean_tel)
    final_accuracy.append(mean_acc)
    
    if (e+1) % 1 == 0:
        T.save(model.state_dict(), f'checkpoint_{e+1}.pth')

    #accuracies.append(acc)
    print(f'Epoch: {e + 1} | loss: {mean_trl} | test loss: {mean_tel} | accuracy:{mean_acc} ')